The point of this script is to gather all the ways that we could make numpy arrays for FES calculations. Generally, these take quite a long time and only need to be run once.

This is loosely based on `parse_gate_COM.py` and `gate_COM_xvg.sh` which I used in previous string sims to get the gate COM out of string sims using other CVs


**The shape of these CVs must be**: `(n_iterations*n_swarms_per_iter*n_beads, n_frames_per_iter, n_cvs)`

In the first dimension, this is also organized in a certain way... Iteration by iteration, then bead by bead, then swarm by swarm

ex: 20 iterations, 10 beads, 3 swarms/ bead = (600,2,2) for 2 cvs

iteration1, bead1, swarm1

iteration1, bead1, swarm2...

iteration19, bead10, swarm3

iteration20, bead1, swarm1,

iteration20, bead1, swarm2

---

Generally, we have been using 16 beads, with 2 fixed ends, so the calculated CVs will be for **14 beads**, and we have also used **32 swarms**


Files are formatted as: `md/$iteration/$bead/$swarm/traj_comp.xtc`

Thankfully, these xtc files should already be 2 frames only, so `n_frames_per_iter` should stay as 2

---


The calculations possible are:
- Salt bridge distances
- Using Darko's CVs

In [1]:
import numpy as np

# Salt bridges

Salt bridge contacts in the simulations are:

**Bridge 1**:
  - E145+R91 always in contact, OF or IF (N term, TM 4 and 3) ( = E151,R97,R407 in paper)
  - R401 contacts when OF (TM11)
  
  
**Bridge 2:**
  - R334+E394 always in contact, OF or IF (C term, TM9 and 10) ( = R340,E400,E336,R158 in paper)
  - R152 contacts when OF(TM5)
  - E330 is on TM 8, might also coordinate
  

# Darko's contacts


Here we load the original np arrays, parse out which frames we are going to read (so that it matches MSM), and then multiply each cv by their score


Also, need to shape it as the MSM dictates, so:
`(n_iterations*n_swarms_per_iter*n_beads, n_frames_per_iter, n_cvs)`

This requires that you take every other frame from the cv and then vstack them together, so this is why I did this

In [2]:
scores_cv1 = {'OUTWARD_OPEN':[0.2394941214441451, 0.17863718195954076, 0.22829366907512316,\
                              0.19946214152438158, 0.15411288599680942],

             'OUTWARD_OCCLUDED':[0.36805920033218326, 0.2764556145667593, 0.3554851851010575],

             'INWARD_OCCLUDED':[0.488214651736396, 0.5117853482636041]}


scores_cv2 = {'OUTWARD_OPEN':[0.2229487221925901, 0.22352902476893688, 0.1541859413566841, \
                              0.15980437637399936, 0.23953193530778957],

             'OUTWARD_OCCLUDED':[0.140953252458209, 0.1772870039340966, 0.13652678227951054, \
                                 0.116917466261338, 0.11464804932410656, 0.17362681856969972, 0.14004062717303947],

             'INWARD_OCCLUDED':[0.1294970260086535, 0.1155039966361535, 0.1255693377498144, 0.11155992278130479,\
                                0.08760322976066104, 0.09880977669901195, 0.10682457260253515,\
                                0.12239235626633586, 0.10223978149552974]}

In [3]:
def parse_cv_groups(start_iteration, end_iteration, cv_group, cv_name):
    cv1 = np.load(f"{indir}/{cv_name}_cv1.npy")
    cv2 = np.load(f"{indir}/{cv_name}_cv2.npy")


    cv1 = cv1[start_iteration:(end_iteration+1)]
    cv2 = cv2[start_iteration:(end_iteration+1)]

    # flatten across the n_iterations * n_frames
    cv1 = np.vstack(cv1)
    cv2 = np.vstack(cv2)
    
    cv1_scored = np.zeros(np.shape(cv1))
    cv2_scored = np.zeros(np.shape(cv2))

    for n, score in enumerate(scores_cv1[cv_group]):
        cv1_scored[:,n] = cv1[:,n] * score

    for n, score in enumerate(scores_cv2[cv_group]):
        cv2_scored[:,n] = cv2[:,n] * score    
    
    cv1_scored = np.sum(cv1_scored, axis = 1)
    cv2_scored = np.sum(cv2_scored, axis = 1)
    
    
    cv1_scored = np.vstack((cv1_scored[::2], cv1_scored[1::2])).T
    cv2_scored = np.vstack((cv2_scored[::2], cv2_scored[1::2])).T

    np.save(f"{indir}/{cv_name}_cv1_scored.npy", cv1_scored)
    np.save(f"{indir}/{cv_name}_cv2_scored.npy", cv2_scored)
    
    return cv1, cv2, cv1_scored, cv2_scored

In [6]:
indir = '../textfiles_out/darko_dists_np/influx_apo_gate_CV'

start_iteration = 99
end_iteration = 745

cv1, cv2, cv1_scored, cv2_scored = parse_cv_groups(start_iteration=start_iteration, 
                end_iteration=end_iteration,
               cv_group='OUTWARD_OPEN',
               cv_name='OutOpen')

In [18]:
np.shape(cv1_scored)

(202944, 2)

In [19]:
cv2_scored[:2]

array([[5.40607594, 5.41188917],
       [5.40607594, 5.20192794]])

In [20]:
cv2

array([[ 3.84916979,  4.18894958,  6.4026099 , ...,  8.63069496,
         3.22114458,  4.70978199],
       [ 4.28931378,  4.06607943,  6.27501566, ...,  8.61730358,
         3.5085874 ,  4.56336326],
       [ 3.84916979,  4.18894958,  6.4026099 , ...,  8.63069496,
         3.22114458,  4.70978199],
       ...,
       [ 3.56983006,  4.8306326 , 17.4263315 , ...,  7.94360957,
         3.05249274,  2.51813278],
       [ 3.35244927,  5.20801417, 16.87741798, ...,  7.96910837,
         3.42795792,  4.10827318],
       [ 2.80784512,  4.73752181, 16.69034975, ...,  7.82131836,
         2.95502826,  4.13948019]])

In [91]:
80/2.5

32.0

In [36]:
OutOcc_cv1 = np.load(f"{indir}/OutOcc_cv1.npy")
OutOcc_cv2 = np.load(f"{indir}/OutOcc_cv2.npy")

In [ ]:

n_iterations = end_iteration - start_iteration + 2
distance_arr_cv1 = np.zeros((len(u.trajectory), len(contacts_cv1_groups[cv_group])))
distance_arr_cv2 = np.zeros((len(u.trajectory), len(contacts_cv2_groups[cv_group])))

## CV1
distance_index = 0
for contact1,contact2 in contacts_cv1_groups[cv_group]:
    resid1,atomid1 = contact1.split('_')
    resid2,atomid2 = contact2.split('_')

    r1 = u.select_atoms(f"resid {int(resid1) - 1} and name {atomid1}") #MDA HAS 0 INDEXING, MINUS 1!!
    r2 = u.select_atoms(f"resid {int(resid2) - 1} and name {atomid2}") #MDA HAS 0 INDEXING

    for time_index, frame in enumerate(u.trajectory):
        distance_arr_cv1[time_index, distance_index] = d.dist(r1, r2)[2]

    distance_index = distance_index + 1



## CV2
distance_index = 0
for contact1,contact2 in contacts_cv2_groups[cv_group]:
    resid1,atomid1 = contact1.split('_')
    resid2,atomid2 = contact2.split('_')

    r1 = u.select_atoms(f"resid {int(resid1) - 1} and name {atomid1}") #MDA HAS 0 INDEXING, MINUS 1!!
    r2 = u.select_atoms(f"resid {int(resid2) - 1} and name {atomid2}") #MDA HAS 0 INDEXING

    for time_index, frame in enumerate(u.trajectory):
        distance_arr_cv2[time_index, distance_index] = d.dist(r1, r2)[2]

    distance_index = distance_index + 1


return distance_arr_cv1, distance_arr_cv2

In [16]:
import MDAnalysis as mda
import MDAnalysis.analysis.distances as d

In [36]:
u = mda.Universe('../confout_files/tpr_files/influx_apo_gate_CV.wholesys.tpr', '../../string_sims/TMD_initial_path/influx_apo_gate_CV/md/1/test.xtc')

In [37]:
#465_HA','68_HG13'

r1 = u.select_atoms("resid 464 and name HA")
r2 = u.select_atoms("resid 67 and name HG13")

protein = u.select_atoms("protein")

In [38]:
for ts in u.trajectory:
    if ts.frame < 10:
        #protein.unwrap(compound='fragments')
        print(d.dist(r1,r2, box = ts.dimensions))
    else:
        break

[[464.        ]
 [ 67.        ]
 [ 37.78642467]]
[[464.        ]
 [ 67.        ]
 [ 37.26870936]]
[[464.        ]
 [ 67.        ]
 [ 37.78642467]]
[[464.       ]
 [ 67.       ]
 [ 37.7781208]]
[[464.        ]
 [ 67.        ]
 [ 37.78642467]]
[[464.        ]
 [ 67.        ]
 [ 38.07990844]]
[[464.        ]
 [ 67.        ]
 [ 37.78642467]]
[[464.       ]
 [ 67.       ]
 [ 38.0316567]]
[[464.        ]
 [ 67.        ]
 [ 37.78642467]]
[[464.        ]
 [ 67.        ]
 [ 38.29621978]]


In [26]:
ts = u.trajectory[1]
ts.dimensions

array([ 94.3472 ,  94.3472 , 113.78223,  90.     ,  90.     ,  90.     ],
      dtype=float32)

In [31]:
ts.frame

759